In [3]:
import pandas

prices_round_1_day__1 = pandas.read_csv("data/island-data-bottle-round-1/prices_round_1_day_-1.csv", delimiter=";")

In [6]:
prices_round_1_day__1

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-1,0,PEARLS,10002,1,9996.0,2.0,9995.0,29.0,10004,2,10005.0,29.0,NaN,NaN,10003.0,0.000000
1,-1,0,BANANAS,4951,1,4946.0,2.0,4945.0,29.0,4952,31,NaN,NaN,NaN,NaN,4951.5,0.000000
2,-1,100,PEARLS,9996,2,9995.0,22.0,NaN,NaN,10004,2,10005.0,22.0,NaN,NaN,10000.0,4.000000
3,-1,100,BANANAS,4945,24,NaN,NaN,NaN,NaN,4952,24,NaN,NaN,NaN,NaN,4948.5,3.444336
4,-1,200,BANANAS,4945,20,NaN,NaN,NaN,NaN,4952,20,NaN,NaN,NaN,NaN,4948.5,3.795410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-1,999700,PEARLS,9995,26,NaN,NaN,NaN,NaN,10005,26,NaN,NaN,NaN,NaN,10000.0,-760.000000
19996,-1,999800,BANANAS,4891,20,NaN,NaN,NaN,NaN,4896,10,4898.0,20.0,NaN,NaN,4893.5,222096.000000
19997,-1,999800,PEARLS,9995,20,NaN,NaN,NaN,NaN,10002,10,10005.0,20.0,NaN,NaN,9998.5,-760.000000
19998,-1,999900,BANANAS,4892,31,NaN,NaN,NaN,NaN,4898,31,NaN,NaN,NaN,NaN,4895.0,216680.000000


In [43]:
trading_matrix = \
[[1,    0.5,    1.45,   0.75],
 [1.95, 1,      3.1,    1.49],
 [0.67, 0.31,   1,      0.48],
 [1.34, 0.64,   1.98,   1]]
# from is on the right
# to is on the top

labels = ['pizza_slice', 'wasabi_root', 'snowball', 'shells']

1.34 * 0.5 * 3.1 * 0.67 * 0.75

1.0436925000000001

In [78]:
import json, sys, math
import networkx as nx

def parse_points(matrix, labels, add_num=0, iteration=1):
    '''
    Parameters:
    matrix - 2D list (m, m) of conversion rates between products
    labels - 1D list (m,) of product labels 
    '''
    edges = []

    for it in range(iteration):
        for jt in range(iteration):
            for i in range(len(labels)): # for row in matrix
                for j in range(len(labels)): # for column in matrix
                    if i != j:
                        edges.append((labels[i] + str(it), labels[j] + str(jt), -1.0 * math.log(float(matrix[i][j])) + add_num))
    return edges

def build_graph(parsed_points):
    dg = nx.DiGraph()
    dg.add_weighted_edges_from(parsed_points)
    return dg

def find_path(digraph, start="shells"):
    path = nx.single_source_bellman_ford(digraph, start)
    return path

def output_path(path, g, start="shells"):
    visited = set(start)
    tot=1.0
    pred = path[start]
    x = start
    while pred not in visited:
        print(pred, "-->", x, math.exp(-g[pred][x]['weight']))
        tot*=math.exp(-g[pred][x]['weight'])
        visited.add(pred)
        x = pred
        pred = path[pred]
    
    tot *= math.exp(-g[start][x]['weight'])
    print(start, "-->", x, math.exp(-g[start][x]['weight']))
    print("Total:", tot)
    if tot < 1.0:
        print("Note: no arbitrage opportunity detected.")

parsed_points = parse_points(trading_matrix, labels, add_num=0, iteration=2)
dg = build_graph(parsed_points)
print(parsed_points)
# path = nx.find_negative_cycle(dg, source='pizza_slice0')
path = nx.single_source_shortest_path(dg, source='shells0')
print(path['shells0'])
print(path['shells1'])
print(nx.shortest_path(dg, source='shells0', target='shells1', weight='weight'))
# path = find_path(dg)
# output_path(path, dg)

[('pizza_slice0', 'wasabi_root0', 0.6931471805599453), ('pizza_slice0', 'snowball0', -0.371563556432483), ('pizza_slice0', 'shells0', 0.2876820724517809), ('wasabi_root0', 'pizza_slice0', -0.6678293725756554), ('wasabi_root0', 'snowball0', -1.1314021114911006), ('wasabi_root0', 'shells0', -0.3987761199573678), ('snowball0', 'pizza_slice0', 0.40047756659712525), ('snowball0', 'wasabi_root0', 1.171182981502945), ('snowball0', 'shells0', 0.7339691750802004), ('shells0', 'pizza_slice0', -0.29266961396282004), ('shells0', 'wasabi_root0', 0.4462871026284195), ('shells0', 'snowball0', -0.6830968447064438), ('pizza_slice0', 'wasabi_root1', 0.6931471805599453), ('pizza_slice0', 'snowball1', -0.371563556432483), ('pizza_slice0', 'shells1', 0.2876820724517809), ('wasabi_root0', 'pizza_slice1', -0.6678293725756554), ('wasabi_root0', 'snowball1', -1.1314021114911006), ('wasabi_root0', 'shells1', -0.3987761199573678), ('snowball0', 'pizza_slice1', 0.40047756659712525), ('snowball0', 'wasabi_root1', 

ValueError: Contradictory paths found: negative weights?

In [77]:
sum = dg['shells0']['pizza_slice0']['weight'] + \
dg['pizza_slice0']['wasabi_root0']['weight'] + \
dg['wasabi_root0']['snowball0']['weight'] + \
dg['snowball0']['pizza_slice1']['weight'] + \
dg['pizza_slice1']['shells1']['weight'] 

print(sum)
print(1/math.exp(sum))

sum = dg['shells0']['pizza_slice0']['weight'] + \
dg['pizza_slice1']['shells1']['weight'] 

print(sum)
print(1/math.exp(sum))

-0.042764905845069257
1.0436925000000001
-0.00498754151103914
1.0050000000000001
